In [1]:
import os
import nltk
import requests
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

In [2]:
nltk.download("punkt")
nltk.download("stopwords")

API_URL = "https://router.huggingface.co/hf-inference/models/facebook/bart-large-cnn"
HEADERS = {"Authorization": f"Bearer {os.environ['HF_TOKEN']}"}

STOP_WORDS = set(stopwords.words("english"))

[nltk_data] Downloading package punkt to /home/sandeep/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sandeep/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
def extractive_summary(text, ratio=1.2):
    words = word_tokenize(text)
    filtered = [w.lower() for w in words if w.isalnum() and w.lower() not in STOP_WORDS]

    freq_table = {}
    for w in filtered:
        freq_table[w] = freq_table.get(w, 0) + 1

    sentences = sent_tokenize(text)
    sentence_scores = {}
    for sentence in sentences:
        for word, freq in freq_table.items():
            if word in sentence.lower():
                sentence_scores[sentence] = sentence_scores.get(sentence, 0) + freq

    if not sentence_scores:
        return text

    avg_score = sum(sentence_scores.values()) / len(sentence_scores)
    summary = " ".join([s for s in sentences if sentence_scores.get(s, 0) > ratio * avg_score])
    return summary if summary.strip() else text

In [11]:
def chunk_text(text, max_words=400):
    words = text.split()
    for i in range(0, len(words), max_words):
        yield " ".join(words[i:i+max_words])

In [7]:
def abstractive_summary(text, min_length=40, max_length=150):
    payload = {"inputs": text, "parameters": {"min_length": min_length, "max_length": max_length, "do_sample": False}}
    try:
        response = requests.post(API_URL, headers=HEADERS, json=payload, timeout=60)
        return response.json()[0]["summary_text"]
    except:
        return text

In [16]:
def hybrid_summarize(text, extract_ratio=1.2, chunk_size=500, min_len=120, max_len=200):
    extract = extractive_summary(text, ratio=extract_ratio)
    final_summaries = []
    for chunk in chunk_text(extract, max_words=chunk_size):
        summary = abstractive_summary(chunk, min_length=min_len, max_length=max_len)
        final_summaries.append(summary)
# Optionally, summarize the combined summaries
    combined_summary = " ".join(final_summaries)
    if len(final_summaries) > 1:
        combined_summary = abstractive_summary(combined_summary, min_length=min_len, max_length=max_len)
    return combined_summary

In [17]:

input_text = """
Quantum computing represents one of the most profound technological advances of the 21st century, with the potential to transform multiple fields including cryptography, artificial intelligence, materials science, and complex optimization problems. Unlike classical computers, which operate on bits in deterministic states of 0 or 1, quantum computers utilize qubits that exploit principles such as superposition and entanglement, enabling the simultaneous exploration of multiple computational pathways. This capability allows quantum machines to solve certain classes of problems exponentially faster than classical counterparts, potentially addressing challenges that have remained intractable for decades. Despite the promise, the development of scalable, fault-tolerant quantum computers faces formidable engineering hurdles. Qubits are highly susceptible to decoherence caused by environmental interference, and error correction schemes often require the use of thousands of physical qubits to construct a single logical qubit. Different physical implementations—ranging from superconducting circuits to trapped ions and photonic systems—offer varying trade-offs in terms of scalability, stability, and operational complexity. Current quantum processors operate at cryogenic temperatures, necessitating sophisticated cooling infrastructures and extremely precise control mechanisms. Beyond hardware, the creation of efficient quantum algorithms is crucial, as naive translation of classical algorithms often fails to harness the full potential of quantum architectures. Fields such as cryptography are particularly affected, with quantum computing threatening to undermine classical public-key cryptosystems like RSA and ECC, prompting research into post-quantum cryptography. In drug discovery and molecular simulation, quantum computers can model complex chemical interactions at atomic resolutions, potentially accelerating the development of new therapeutics and materials. However, practical applications are currently constrained by limited qubit counts and coherence times, requiring hybrid approaches that combine classical and quantum computing. Despite these limitations, experimental demonstrations of quantum supremacy have shown that quantum devices can outperform classical computers on specific, well-defined tasks. As research continues, the integration of quantum computing into mainstream technology could redefine computational problem-solving, influence global security paradigms, and catalyze breakthroughs across scientific disciplines. The trajectory of this field is highly dynamic, demanding interdisciplinary expertise in physics, computer science, engineering, and mathematics to address both theoretical and practical challenges. The ultimate success of quantum computing will depend not only on technological advancements but also on ethical, economic, and societal considerations surrounding accessibility, control, and the responsible deployment of transformative computational capabilities.
"""  
summary = hybrid_summarize(input_text, extract_ratio=1.2)
print(summary)

Quantum computing represents one of the most profound technological advances of the 21st century. Unlike classical computers, which operate on bits in deterministic states of 0 or 1, quantum computers utilize qubits that exploit principles such as superposition and entanglement. Quantum computing has the potential to transform multiple fields including cryptography, artificial intelligence, materials science, and complex optimization problems. For more information, visit quantum-computing.org and the University of Washington ’s Quantum Computing Center.’ for more information on the University of Washington’ ‘Quantum Computing Center’, click here.
